In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize as sklearn_normalize
from sklearn.utils import shuffle
from tensorflow import keras

np.random.seed(1)

In [ ]:
def normalize(x):
    return (np.array(x) - 0) / (3000 - 0)

## Load data and make predictions

In [ ]:
train_data_dir = '../data/training_sites'

with open(os.path.join(train_data_dir, "negative_data.pkl"), 'rb') as file:
    x_negative = np.array(pickle.load(file))
file.close()
y_negative = np.zeros(len(x_negative))

with open(os.path.join(train_data_dir, "positive_data.pkl"), 'rb') as file:
    x_positive = np.array(pickle.load(file))
file.close()

with open(os.path.join(train_data_dir, "positive_data_test.pkl"), 'rb') as file:
    x_positive_test = np.array(pickle.load(file))
file.close()

In [ ]:
ndvi_train = (x_positive[:,7] - x_positive[:,3]) / (x_positive[:,7] + x_positive[:,3])
ndvi_test = (x_positive_test[:,7] - x_positive_test[:,3]) / (x_positive_test[:,7] + x_positive_test[:,3])

lower_bound = -0.1
upper_bound = 0.5
index_train = np.logical_and(ndvi_train > lower_bound, ndvi_train < upper_bound)
index_test = np.logical_and(ndvi_test > lower_bound, ndvi_test < upper_bound)

x_positive = x_positive[index_train]
y_positive = np.ones(len(x_positive))
x_positive_test = x_positive_test[index_test]
y_positive_test = np.ones(len(x_positive_test))

In [ ]:
x = np.concatenate((x_positive, x_negative))
y = np.concatenate((y_positive, y_negative))

x, y = shuffle(x, y, random_state=42)
x = normalize(x)
x_positive_test = normalize(x_positive_test)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
print("Num Train:\t\t", len(x_train))
print("Num Test:\t\t", len(x_test))
print(f"Percent Negative Train:\t {100 * sum(y_train == 0.0) / len(y_train):.1f}")
print(f"Percent Negative Test:\t {100 * sum(y_test == 0.0) / len(y_test):.1f}")

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
x_positive_test = np.expand_dims(x_positive_test, -1)

num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_positive_test = keras.utils.to_categorical(y_positive_test, num_classes)

In [ ]:
model = keras.models.load_model('../models/model_filtered-12-07-2020.h5')

In [ ]:
num_test_set_samples = 5000
random_test_indices = np.random.choice(len(x_test), num_test_set_samples)

inputs = np.concatenate((x_positive_test, x_test[random_test_indices]))
preds = model.predict(inputs)
labels = np.concatenate((y_positive_test, y_test[random_test_indices]))

In [ ]:
rgb = []
for sample in inputs:
    sample_rgb = np.zeros((1,1,3))
    sample_rgb[0,0,:] = [sample[3], sample[2], sample[1]]
    rgb.append(sample_rgb)
rgb_min = np.array(rgb).min()
rgb_max = np.array(rgb).max()
rgb = np.array([(np.array(element) - rgb_min) / (rgb_max - rgb_min) for element in rgb])

In [ ]:
def plot_rgb_hist(rgb, gamma, title):
    colors = ['C0', 'red', 'green']
    plt.figure(figsize=(8,5), dpi=100, facecolor=(1,1,1))
    for channel in range(3):
        plt.hist((np.array(rgb)[:,0,0, 2 - channel]) ** gamma, 
                 bins=len(rgb) // 100, 
                 facecolor=colors[channel], 
                 alpha=0.55)
    plt.title(title + ' - Gamma ' + str(gamma))
    plt.show()

In [ ]:
plot_rgb_hist(rgb, 0.5, 'RGB Histogram for Test Set')

## Compile TP, FP, TN, FN classes

In [ ]:
THRESHOLD = 0.90

In [ ]:
def isolate_category(labels, preds, label_class, pred_class):
    class_index = []
    for label, pred in zip(labels, preds):
        binary_pred = pred[1] > THRESHOLD
        label = label[1]
        if label == label_class and binary_pred == pred_class:
            class_index.append(1)
        else:
            class_index.append(0)
    return np.array(class_index).astype('bool')

In [ ]:
tp_index = isolate_category(labels, preds, 1, 1)
tn_index = isolate_category(labels, preds, 0, 0)
fp_index = isolate_category(labels, preds, 0, 1)
fn_index = isolate_category(labels, preds, 1, 0)

print(f"True Positive Rate: {100 * sum(tp_index) / sum(labels[:,1] == 1):.1f}%")
print(f"True Negative Rate: {100 * sum(tn_index) / sum(labels[:,1] == 0):.1f}%")

## Evaluate spectral profiles of classification categories

In [ ]:
# Sentinel 2 band descriptions
band_descriptions = {
    'B1': 'Aerosols, 442nm',
    'B2': 'Blue, 492nm',
    'B3': 'Green, 559nm',
    'B4': 'Red, 665nm',
    'B5': 'Red Edge 1, 704nm',
    'B6': 'Red Edge 2, 739nm',
    'B7': 'Red Edge 3, 779nm',
    'B8': 'NIR, 833nm',
    'B8A': 'Red Edge 4, 864nm',
    'B9': 'Water Vapor, 943nm',
    'B11': 'SWIR 1, 1610nm',
    'B12': 'SWIR 2, 2186nm'
}

band_wavelengths = [442, 492, 559, 665, 704, 739, 779, 833, 864, 943, 1610, 2186]

In [ ]:
num_samples = 1000

plt.figure(figsize=(10,4), dpi=150)

for reflectance in inputs[labels[:,1] == 1][:num_samples]:
    plt.plot(band_wavelengths, reflectance, c='r', alpha=0.01)

for reflectance in inputs[labels[:,1] == 0][:num_samples]:
    plt.plot(band_wavelengths, reflectance, c='C0', alpha=0.01)

plt.xticks(band_wavelengths, band_descriptions.keys(), ha='right', rotation=45, size=8)
plt.xlabel('Band Name')
plt.ylabel('Normalized Reflectance')
plt.ylim([-0.1, 2.0])
    
legend_lines = [Line2D([0], [0], color='r', lw=2),
                Line2D([0], [0], color='C0', lw=2)]
plt.legend(legend_lines, ['Dump Sites', 'Non-Dump Sites'], loc='upper left')

plt.title('Spectral Profiles of Positive and Negative Classes')
plt.show()

In [ ]:
num_samples = 3000
pca_inputs = np.squeeze(np.concatenate((inputs[labels[:,1] == 0][:num_samples], inputs[labels[:,1] == 1][:num_samples])))
pca = TSNE(n_components=2)
pca_out = pca.fit_transform(pca_inputs)
plt.figure(figsize=(8,8), dpi=150)
plt.scatter(pca_out[num_samples:,0], pca_out[num_samples:,1], color='r', s=2, label='Dump Sites', alpha=0.25)
plt.scatter(pca_out[:num_samples,0], pca_out[:num_samples,1], s=2, label='Non-Dump Sites', alpha=0.25)
plt.xticks([])
plt.yticks([])
#plt.ylim([-2, 1.5])
#plt.xlim([-2.5, 2])
legend = plt.legend(markerscale=3)
for element in legend.legendHandles:
    element.set_alpha(1)
plt.title('tSNE Dimensionality Reduction of Hyperspectral Pixels by Class', size=12)
plt.show()

In [ ]:
num_samples = 1000

plt.figure(figsize=(10,4), dpi=150, facecolor=(1,1,1))

for reflectance in inputs[fn_index][:num_samples]:
    plt.plot(band_wavelengths, reflectance, c='r', alpha=0.01)

for reflectance in inputs[tp_index][:num_samples]:
    plt.plot(band_wavelengths, reflectance, c='C0', alpha=0.01)

plt.xticks(band_wavelengths, band_descriptions.keys(), ha='right', rotation=45, size=8)
plt.xlabel('Band Name')
plt.ylabel('Normalized Reflectance')
    
legend_lines = [Line2D([0], [0], color='C0', lw=2),
                Line2D([0], [0], color='r', lw=2)]
plt.legend(legend_lines, ['True Positives', 'False Negatives'], loc='upper left')

plt.title('Spectral Profiles of True Positive and False Negative Samples')
plt.show()

In [ ]:
num_samples = 1000

plt.figure(figsize=(10,4), dpi=150, facecolor=(1,1,1))

for reflectance in inputs[tn_index][:num_samples]:
    plt.plot(band_wavelengths, reflectance, c='C0', alpha=0.01)
    
for reflectance in inputs[fn_index][:num_samples]:
    plt.plot(band_wavelengths, reflectance, c='r', alpha=0.01)

plt.xticks(band_wavelengths, band_descriptions.keys(), ha='right', rotation=45, size=8)
plt.xlabel('Band Name')
plt.ylabel('Normalized Reflectance')
    
legend_lines = [Line2D([0], [0], color='C0', lw=2),
                Line2D([0], [0], color='r', lw=2)]
plt.legend(legend_lines, ['True Negatives', 'False Negatives'], loc='upper left')

plt.title('Spectral Profiles of True Negative and False Negative Samples')
plt.show()

In [ ]:
num_samples = 1000

plt.figure(figsize=(10,4), dpi=150, facecolor=(1,1,1))

for reflectance in inputs[tp_index][:num_samples]:
    plt.plot(band_wavelengths, reflectance, c='C0', alpha=0.01)
    
for reflectance in inputs[fp_index]:
    plt.plot(band_wavelengths, reflectance, c='r', alpha=0.05)

plt.xticks(band_wavelengths, band_descriptions.keys(), ha='right', rotation=45, size=8)
plt.xlabel('Band Name')
plt.ylabel('Normalized Reflectance')
    
legend_lines = [Line2D([0], [0], color='C0', lw=2),
                Line2D([0], [0], color='r', lw=2)]
plt.legend(legend_lines, ['True Positives', 'False Positives'], loc='upper left')

plt.title('Spectral Profiles of True Positive and False Positive Samples')
plt.show()

## What do correctly and incorrectly classified pixels look like?

### Scatterplot representation of category pixel colors

In [ ]:
from sklearn.decomposition import PCA

point_size = 5
gamma = 0.65

plt.figure(figsize=(12,12), dpi=200, facecolor=(1,1,1))
plt.subplot(2,2,1)
pca = PCA(n_components=2)
pca_out = pca.fit_transform(np.squeeze(inputs[tp_index]))
plt.scatter(pca_out[:,0], pca_out[:,1], c=rgb[tp_index][:,0,0,:] ** gamma, s=point_size)
plt.xticks([])
plt.yticks([])
plt.title('True Positive Samples', size=10)

plt.subplot(2,2,2)
pca = PCA(n_components=2)
pca_out = pca.fit_transform(np.squeeze(inputs[fn_index]))
plt.scatter(pca_out[:,0], pca_out[:,1], c=rgb[fn_index][:,0,0,:] ** gamma, s=point_size)
plt.xticks([])
plt.yticks([])
plt.title('False Negative Samples', size=10)

plt.subplot(2,2,3)
pca = PCA(n_components=2)
pca_out = pca.fit_transform(np.squeeze(inputs[fp_index]))
plt.scatter(pca_out[:,0], pca_out[:,1], c=rgb[fp_index][:,0,0,:] ** gamma, s=point_size)
plt.xticks([])
plt.yticks([])
plt.title('False Positive Samples', size=10)


plt.subplot(2,2,4)
pca = PCA(n_components=2)
pca_out = pca.fit_transform(np.squeeze(inputs[tn_index]))
plt.scatter(pca_out[:,0], pca_out[:,1], c=rgb[tn_index][:,0,0,:] ** gamma, s=point_size)
plt.xticks([])
plt.yticks([])
plt.title('True Negative Samples', size=10)

plt.suptitle('PCA Dimensionality Reduction of Pixel Vectors Colored by RGB Values', size=16)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.manifold import TSNE

plt.figure(figsize=(12,12), dpi=200, facecolor=(1,1,1))
plt.subplot(2,2,1)
tsne = TSNE(n_components=2)
tsne_out = tsne.fit_transform(np.squeeze(inputs[tp_index]))
plt.scatter(tsne_out[:,0], tsne_out[:,1], c=rgb[tp_index][:,0,0,:] ** gamma, s=point_size)
plt.xticks([])
plt.yticks([])
plt.title('True Positive Samples', size=10)

plt.subplot(2,2,2)
tsne = TSNE(n_components=2)
tsne_out = tsne.fit_transform(np.squeeze(inputs[fn_index]))
plt.scatter(tsne_out[:,0], tsne_out[:,1], c=rgb[fn_index][:,0,0,:] ** gamma, s=point_size)
plt.xticks([])
plt.yticks([])
plt.title('False Negative Samples', size=10)

plt.subplot(2,2,3)
tsne = TSNE(n_components=2)
tsne_out = tsne.fit_transform(np.squeeze(inputs[fp_index]))
plt.scatter(tsne_out[:,0], tsne_out[:,1], c=rgb[fp_index][:,0,0,:] ** gamma, s=point_size)
plt.xticks([])
plt.yticks([])
plt.title('False Positive Samples', size=10)


plt.subplot(2,2,4)
tsne = TSNE(n_components=2)
tsne_out = tsne.fit_transform(np.squeeze(inputs[tn_index]))
plt.scatter(tsne_out[:,0], tsne_out[:,1], c=rgb[tn_index][:,0,0,:] ** gamma, s=point_size)
plt.xticks([])
plt.yticks([])
plt.title('True Negative Samples', size=10)

plt.suptitle('tSNE Dimensionality Reduction of Pixel Vectors Colored by RGB Values', size=16)
plt.tight_layout()
plt.show()

### Visualize pixel colors directly

In [ ]:
def plot_pixel_grid(preds, colors, title):
    plt.figure(figsize=(20,20), dpi=50, facecolor=(1,1,1))
    num_samples = np.min([len(preds), 100])
    indices = np.random.choice(len(preds), num_samples)
    for i in range(num_samples):
        index = indices[i]
        pred = preds[index]
        rgb = colors[index] ** gamma
        plt.subplot(10,10,i + 1)
        plt.imshow(rgb)
        plt.axis('off')
        plt.title(f"{pred[1]:.2f}")
    plt.suptitle(title, size=40, y=1.01)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_pixel_grid(preds[tp_index], rgb[tp_index], 'Colors of Randomly Selected True Positives and Prediction Values')

In [ ]:
plot_pixel_grid(preds[tn_index], rgb[tn_index], 'Colors of Randomly Selected True Negatives and Prediction Values')

In [ ]:
plot_pixel_grid(preds[fp_index], rgb[fp_index], 'Colors of Randomly Selected False Positives and Prediction Values')

In [ ]:
plot_pixel_grid(preds[fn_index], rgb[fn_index], 'Colors of Randomly Selected False Negatives and Prediction Values')